# Strategies to tackle context window limitations
Models always have a context window i.e. the length of words, actually **tokens**, it can process at a given time. This is an issue when we want to process any text that doesn't fit in this context window. We can break the text into chunks that can fit. In this notebook we demonstrate a number of techniques to tackle this issue.

In [1]:
text = "As a fundamental problem in organic chemistry, synthesis planning aims at designing energy and cost-efficient reaction pathways for target compounds. In synthesis planning, it is crucial to understand regioselectivity, or the preference of a reaction over competing reaction sites. Precisely predicting regioselectivity enables early exclusion of unproductive reactions and paves the way to designing high-yielding synthetic routes with minimal separation and material costs. However, it is still at emerging state to combine chemical knowledge and data-driven methods to make practical predictions for regioselectivity. At the same time, metal-catalyzed cross-coupling reactions have profoundly transformed medicinal chemistry, and thus become one of the most frequently encountered types of reactions in synthesis planning. In this work, we for the first time introduce a chemical knowledge informed message passing neural network(MPNN) framework that directly identifies the intrinsic major products for metal-catalyzed cross-coupling reactions with regioselective ambiguity. Integrating both first principle methods and data-driven methods, our model achieves an overall accuracy of 95.24\\% on the test set of eight typical metal-catalyzed cross-coupling reaction types, including Suzuki-Miyaura, Stille, Sonogashira, Buchwald-Hartwig, Hiyama, Kumada, Negishi, and Heck reactions, outperforming other commonly used model types. To integrate electronic effects with steric effects in regioselectivity prediction, we propose a quantitative method to measure the steric hindrance effect. Our steric hindrance checker can successfully identify regioselectivity induced solely by steric hindrance. Notably under practical scenarios, our model outperforms 6 experimental organic chemists with an average working experience of over 10 years in the organic synthesis industry in terms of predicting major products in regioselective cases. We have also exemplified the practical usage of our model by fixing routes designed by open-access synthesis planning software and improving reactions by identifying low-cost starting materials. To assist general chemists in making prompt decisions about regioselectivity, we have developed a free web-based AI-empowered tool."

### Fixed size chunking

The simplest approach is to make chunks that fit in your context window without worrying about where it cuts the text. This, as you can see in the example below, is not ideal. Some words get chopped and when the LLM will see these chunks separately, it won't be able to infer information correctly.

In [2]:
chunk_size = 20
[text[i: i + chunk_size] for i in range(0, len(text), chunk_size)]

['As a fundamental pro',
 'blem in organic chem',
 'istry, synthesis pla',
 'nning aims at design',
 'ing energy and cost-',
 'efficient reaction p',
 'athways for target c',
 'ompounds. In synthes',
 'is planning, it is c',
 'rucial to understand',
 ' regioselectivity, o',
 'r the preference of ',
 'a reaction over comp',
 'eting reaction sites',
 '. Precisely predicti',
 'ng regioselectivity ',
 'enables early exclus',
 'ion of unproductive ',
 'reactions and paves ',
 'the way to designing',
 ' high-yielding synth',
 'etic routes with min',
 'imal separation and ',
 'material costs. Howe',
 'ver, it is still at ',
 'emerging state to co',
 'mbine chemical knowl',
 'edge and data-driven',
 ' methods to make pra',
 'ctical predictions f',
 'or regioselectivity.',
 ' At the same time, m',
 'etal-catalyzed cross',
 '-coupling reactions ',
 'have profoundly tran',
 'sformed medicinal ch',
 'emistry, and thus be',
 'come one of the most',
 ' frequently encounte',
 'red types of reactio',


### Splitting based on special characters

We can improve on this and start splitting by special characters such as "." or "\n". This keeps most semantic information in the same chunk. But of course, there can be cases where some information is lost in a previous chunk, sentence when we split by ".".

In [4]:
text.split(".")

['As a fundamental problem in organic chemistry, synthesis planning aims at designing energy and cost-efficient reaction pathways for target compounds',
 ' In synthesis planning, it is crucial to understand regioselectivity, or the preference of a reaction over competing reaction sites',
 ' Precisely predicting regioselectivity enables early exclusion of unproductive reactions and paves the way to designing high-yielding synthetic routes with minimal separation and material costs',
 ' However, it is still at emerging state to combine chemical knowledge and data-driven methods to make practical predictions for regioselectivity',
 ' At the same time, metal-catalyzed cross-coupling reactions have profoundly transformed medicinal chemistry, and thus become one of the most frequently encountered types of reactions in synthesis planning',
 ' In this work, we for the first time introduce a chemical knowledge informed message passing neural network(MPNN) framework that directly identifies the 

### Overlap between chunks

To try to not lose this semantic information too much, we can add some overlap between chunks. This way some information is trickled in from the previous chunk and some from the next. Imagine this as reading the last two lines of the last paragraph and the first two of the next alongside the current paragraph you are reading.

In [5]:
sentences = text.split(".")
overlap = 15
[sentences[i-1][-overlap:] + sentences[i] + sentences[i+1][:5]  for i in range(0, len(sentences)-1)]

['As a fundamental problem in organic chemistry, synthesis planning aims at designing energy and cost-efficient reaction pathways for target compounds In s',
 'arget compounds In synthesis planning, it is crucial to understand regioselectivity, or the preference of a reaction over competing reaction sites Prec',
 ' reaction sites Precisely predicting regioselectivity enables early exclusion of unproductive reactions and paves the way to designing high-yielding synthetic routes with minimal separation and material costs Howe',
 ' material costs However, it is still at emerging state to combine chemical knowledge and data-driven methods to make practical predictions for regioselectivity At t',
 'egioselectivity At the same time, metal-catalyzed cross-coupling reactions have profoundly transformed medicinal chemistry, and thus become one of the most frequently encountered types of reactions in synthesis planning In t',
 'thesis planning In this work, we for the first time introduce a chem

### Embeddings, Vectors, RAG

If there are too many chunks to process all of them every time a query is made, a RAG, Retrieval Augmented Generation, approach can be used. This is usually used with a vector database to do a similarity search to find relevant chunks before querying the LLM.

In [ ]:
!pip install sentence-transformers chromadb

In [7]:
from sentence_transformers import SentenceTransformer

We can use an embedding model to find suitable vectors to represent our vocabulary whether it is words or sentences. These vectors are then stored in a vector database for retrieval later.

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
sentences = text.split(".")

text_embeddings = model.encode(sentences)
print(text_embeddings.shape)

ChromaDB is an Open Source Vector Database that we can use for our RAG application to query before the request is sent to the LLM. ChromaDB as default uses the same Sentence Embedding model we used above, "all-MiniLM-L6-v2".

In [ ]:
import chromadb
client = chromadb.Client()
collection = client.create_collection(name="MySentenceStore")
collection.add(documents=sentences, ids=[str(id) for id in range(0, len(sentences))])

Before we send our query to the LLM, we find a relevant chunk from our vector database. In this example, it will give us the sentence most relevant to our question.

In [10]:
query_results = collection.query(query_texts=["What has transformed medicinal chemistry?"], n_results=1)

In [11]:
print(query_results["documents"])

[[' At the same time, metal-catalyzed cross-coupling reactions have profoundly transformed medicinal chemistry, and thus become one of the most frequently encountered types of reactions in synthesis planning']]
